### 1) Необходимо собрать информацию о вакансиях на вводимую должность (используем input или через аргументы) с сайта superjob.ru и hh.ru. Приложение должно анализировать несколько страниц сайта(также вводим через input или аргументы). Получившийся список должен содержать в себе минимум:

*Наименование вакансии
*Предлагаемую зарплату (отдельно мин. отдельно макс. и отдельно валюту)
*Ссылку на саму вакансию
*Сайт откуда собрана вакансия

По своему желанию можно добавить еще работодателя и расположение. Данная структура должна быть одинаковая для вакансий с обоих сайтов. Общий результат можно вывести с помощью dataFrame через pandas.

In [76]:
from bs4 import BeautifulSoup as bs
import requests
from pprint import pprint
import re
import pandas as pd
from transliterate import translit

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

In [77]:
def parse_hh_compensation(value):
    min_com = None
    max_com = None
    cur = None
    if value:
        match = re.search(r'\b(руб|EUR|USD)\b', value)
        cur = match[0] if match else None
        
        match = re.search(r'\d*\s*\d*-\d*\s*\d*', value)
        if match:
            min_com = match[0].split('-')[0].replace('\xa0', '')
            max_com = match[0].split('-')[1].replace('\xa0', '')
            
        match = re.search(r'от\s\d*\s*\d*', value)
        if match:
            min_com = match[0].replace('\xa0', '').replace('от', '').replace(' ', '')
            
        match = re.search(r'до\s\d*\s*\d*', value)
        if match:
            max_com = match[0].replace('\xa0', '').replace('до', '').replace(' ', '')
    
    return {'min': min_com, 'max': max_com, 'cur': cur}

In [78]:
def hh_vacancy(query, page = 0):
    hh_link = 'https://hh.ru'
    params = {'st':'searchVacancy', 'text':query, 'page': page}
    headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.138 Safari/537.36'}
    html = requests.get(hh_link+'/search/vacancy', params=params, headers=headers).text
    soup = bs(html, 'lxml')
    vacancy_block = soup.find('div', {'class':'vacancy-serp'})
    if vacancy_block is None:
        return []
    vacancy_list = vacancy_block.findChildren('div', {'class':'vacancy-serp-item'}, recursive=False)
    
    items = []
    for item in vacancy_list:
        data = {}
        header = item.find('div', {'class':'vacancy-serp-item__row_header'})
        compensation = header.find('div', {'class':'vacancy-serp-item__sidebar'}).find('span', {'class':'bloko-section-header-3'})
        compensation = parse_hh_compensation(compensation.getText() if compensation else None)
        data['title'] = header.find('span', {'class':'g-user-content'}).getText()
        data['compensation_min'] = compensation['min']
        data['compensation_max'] = compensation['max']
        data['compensation_cur'] = compensation['cur']
        data['link'] = header.find('a')['href']
        data['site'] = hh_link
        items.append(data)
    return items

In [79]:
def superjob_vacancy(query, page = 1):
    trans_query = translit(query, "ru", reversed=True).lower()
    superjob_link = 'https://russia.superjob.ru'
    offset = page * 20
    limit = 20
    params = {'page': page}
    headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.138 Safari/537.36'}
    html = requests.get(superjob_link+'/vakansii/'+ trans_query +'.html', params=params, headers=headers).text
    soup = bs(html, 'lxml')
    vacancy_block = soup.find('div', {'class':'f-test-vacancy-item'})
    if vacancy_block is None:
        return []
    vacancy_block = vacancy_block.find_parent()
    vacancy_list = vacancy_block.findChildren('div', {'class':'f-test-vacancy-item'}, recursive=False)
    
    items = []
    for item in vacancy_list:
        data = {}        
        title = item.find('a').getText()        
        compensation = item.find('a').find_parent().find_next_sibling()
        compensation = parse_hh_compensation(compensation.getText())
        data['title'] = title
        data['compensation_min'] = compensation['min']
        data['compensation_max'] = compensation['max']
        data['compensation_cur'] = compensation['cur']
        data['link'] = superjob_link + item.find('a')['href']
        data['site'] = superjob_link
        items.append(data)
    return items

In [80]:
query = None
page = None
while(True):
    if query is None:
        query = input('Укажите вакансию для поиска: ')
    if page is None:
        page = 1
    else:
        page = int(input('Укажите страницу: '))
    items = hh_vacancy(query, page - 1) + superjob_vacancy(query, page)
    if len(items):
        vacancy = pd.DataFrame(data=items)
        print(vacancy)
    else:
        print('Вакансии не найдены или страницы не существует')

Укажите вакансию для поиска: программист
                                                              title  \
0   Front-End Developer (Vue.js)                                      
1   Developer Drupal / Программист Drupal                             
2   Ruby on Rails Developer                                           
3   C# разработчик (начинающий, junior)                               
4   Developer Java / Программист Java (Spring Boot, VueJS)            
5   Developer Java + Perl/ Программист Java + Perl                    
6   Backend разработчик (middle backend php developer)                
7   Magento e-commerce lead developer / Ведущий программист Magento   
8   Программист роботов                                               
9   Java-разработчик на "Единый клиент"                               
10  PHP-программист                                                   
11  Программист Unity / Unity Developer                               
12  Unity C# Developer / Разработчик

Укажите страницу: 4
                                               title compensation_min  \
0   Начинающий Erlang/Elixir разработчик              100000            
1   Программист 1C                                    120000            
2   Embedded программист-разработчик                  100000            
3   Программист- разработчик 1С                       130000            
4   Программист Axapta                                150000            
5   Программист Axapta                                150000            
6   Программист                                       70000             
7   PHP программист                                   1200              
8   Инженер-программист АСУ ТП                        30000             
9   Backend-разработчик                               90000             
10  Программист (C, Linux)                            100000            
11  Ведущий веб-разработчик                           200000            
12  Начинающий Erlang/Elixir ра

Укажите страницу: 5


KeyboardInterrupt: 